In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-risk-dataset/LCDataDictionary.xlsx
/kaggle/input/credit-risk-dataset/loan/loan.csv


# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex

sns.set_style('whitegrid')

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score

# Statistical Data Analysis

In [3]:
#loading the dataset

df_loan = pd.read_csv("/kaggle/input/credit-risk-dataset/loan/loan.csv")


/tmp/ipykernel_19/1047268947.py:3: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loan = pd.read_csv("/kaggle/input/credit-risk-dataset/loan/loan.csv")


In [4]:
df_loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_loan.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [6]:
def columns_types(dataset):
    #columns_types function will print different dtypes available in all the columns with the counts
    int_col = len(dataset.select_dtypes('int').columns)
    float_col = len(dataset.select_dtypes('float').columns)
    object_col = len(dataset.select_dtypes('object').columns)
    print('There are {} int, {} float and {} object type features available in the dataset'.format(int_col, float_col, object_col-1))

columns_types(df_loan)

There are 2 int, 49 float and 22 object type features available in the dataset


In [7]:
# summary table function
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [8]:
summary(df_loan).style.background_gradient(cmap='YlOrBr')

data shape: (887379, 74)


,data type,#missing,%missing,#unique,min,max,average,standard_deviation,first value,second value,third value
id,int64,0,0.000000,887379,54734.000000,68617057.000000,32465133.055024,22827341.721266,1077501,1077430,1077175
member_id,int64,0,0.000000,887379,70473.000000,73544841.000000,35001824.815562,24113354.675557,1296599,1314167,1313524
loan_amnt,float64,0,0.000000,1372,500.000000,35000.000000,14755.264605,8435.455601,5000.000000,2500.000000,2400.000000
funded_amnt,float64,0,0.000000,1372,500.000000,35000.000000,14741.877625,8429.897657,5000.000000,2500.000000,2400.000000
funded_amnt_inv,float64,0,0.000000,9856,0.000000,35000.000000,14702.464383,8442.106732,4975.000000,2500.000000,2400.000000
term,object,0,0.000000,2,nan,nan,nan,nan,36 months,60 months,36 months
int_rate,float64,0,0.000000,542,5.320000,28.990000,13.246740,4.381867,10.650000,15.270000,15.960000
installment,float64,0,0.000000,68711,15.670000,1445.460000,436.717127,244.186593,162.870000,59.830000,84.330000
grade,object,0,0.000000,7,nan,nan,nan,nan,B,C,C
sub_grade,object,0,0.000000,35,nan,nan,nan,nan,B2,C4,C5


From the table we can see a lot of columns with missing percentage more than 97% and we will drop those columns in data preprocessing.

# Data Preprocessing

In [9]:
#here we are eliminating columns having missing values 97% and more.
for col in df_loan.columns:
    if ((df_loan[col].isna().sum()/len(df_loan))*100) >= 97:
        df_loan = df_loan.drop([col], axis=1)
        
columns_types(df_loan)

There are 2 int, 33 float and 21 object type features available in the dataset


In [10]:
df_loan

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,"5,000.00","5,000.00","4,975.00",36 months,10.65,162.87,B,B2,...,NaN,Jan-2016,0.00,NaN,1.00,INDIVIDUAL,0.00,NaN,NaN,NaN
1,1077430,1314167,"2,500.00","2,500.00","2,500.00",60 months,15.27,59.83,C,C4,...,NaN,Sep-2013,0.00,NaN,1.00,INDIVIDUAL,0.00,NaN,NaN,NaN
2,1077175,1313524,"2,400.00","2,400.00","2,400.00",36 months,15.96,84.33,C,C5,...,NaN,Jan-2016,0.00,NaN,1.00,INDIVIDUAL,0.00,NaN,NaN,NaN
3,1076863,1277178,"10,000.00","10,000.00","10,000.00",36 months,13.49,339.31,C,C1,...,NaN,Jan-2015,0.00,NaN,1.00,INDIVIDUAL,0.00,NaN,NaN,NaN
4,1075358,1311748,"3,000.00","3,000.00","3,000.00",60 months,12.69,67.79,B,B5,...,Feb-2016,Jan-2016,0.00,NaN,1.00,INDIVIDUAL,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887374,36371250,39102635,"10,000.00","10,000.00","10,000.00",36 months,11.99,332.10,B,B5,...,Feb-2016,Jan-2016,0.00,NaN,1.00,INDIVIDUAL,0.00,0.00,"25,274.00","17,100.00"
887375,36441262,39152692,"24,000.00","24,000.00","24,000.00",36 months,11.99,797.03,B,B5,...,Feb-2016,Jan-2016,0.00,29.00,1.00,INDIVIDUAL,0.00,0.00,"140,285.00","10,200.00"
887376,36271333,38982739,"13,000.00","13,000.00","13,000.00",60 months,15.99,316.07,D,D2,...,Feb-2016,Jan-2016,0.00,NaN,1.00,INDIVIDUAL,0.00,0.00,"34,178.00","18,000.00"
887377,36490806,39222577,"12,000.00","12,000.00","12,000.00",60 months,19.99,317.86,E,E3,...,Feb-2016,Jan-2016,1.00,22.00,1.00,INDIVIDUAL,0.00,0.00,"58,418.00","27,000.00"


We can see than the column number has changed from 74 to 57. All the columns with 97% or more missing values have been dropped.